In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Optimize Graph Coloring

Graph coloring is a fundamental NP-complete problem with many applications in, e.g., schedule planning and map coloring. For testing different and standardized graphs, please download the graph coloring instances from [https://mat.tepper.cmu.edu](https://mat.tepper.cmu.edu/COLOR/instances.html) ([tar file](https://mat.tepper.cmu.edu/COLOR/instances/instances.tar)). You can load them in the Optimizer in the last cell using `Mode: File`. Note: We do not support loading binary files ending with `.b`, only files ending with `.col` can be loaded. 

## Problem and QUBO model
Let $V$ be the set of $N$ vertices and $E$ the set of edges in a given graph. The colors can be chosen from the set $C$, containing $K$ colors. In Graph Coloring we seek a mapping $f: V \to C$ from the set of vertices $V$ into the set of colors $C$ such that for adjacent vertices $v, w \in V$, i.e., $(v,w) \in E$, the vertices have different colors $f(v) \neq f(w)$.

The bit $x_{n,k}=1$ if the vertex $n$ is colored with $k$. The QUBO can then be written as:
$$
H = A H_1 + B H_2 = A \sum_{n=0}^{N-1}\left(1-\sum_{k=0}^{K-1}x_{n,k}\right)^2 + B \sum_{(n,m)\in E}\sum_{k=0}^{K-1}x_{n,k}x_{m,k}
$$

This workbook defines a simple ``OptimizerModel`` for the graph coloring problem. It can be used with the Optimization to tune parameters.

## Implementation
This workbook defines a simple OptimizerModel for the graph coloring problem. It can be used to tune parameters. Run the following block to define the class `GraphColoring`.

In [ ]:
import matplotlib.pyplot as plt
from dadk.Optimizer import *
import networkx as nx
from random import random, seed

def load_graph(filename:str)->nx.Graph:
    print(f"loading {filename:32s} ... ", end='')
    with open(filename) as fp:
        for line_no, line in enumerate(fp.readlines()):
            line = str(line).strip().replace('  ', ' ').replace('\t', ' ')
            while '  ' in line:
                line = str(line).strip().replace('  ', ' ').replace('\t', ' ')
            if line == '' or line.startswith('c'):
                continue
            elif line.startswith('p'):
                parts = line.split(' ')
                if parts[0] == 'p' and parts[1] in ['edge', 'col', 'edges']:
                    graph = nx.Graph()
                    for n in range(int(parts[2])):
                        graph.add_node(n)
            elif line.startswith('e'):
                parts = line.split(' ')
                if parts[0] == 'e':
                    graph.add_edge(int(parts[1]) - 1, int(parts[2]) - 1)
            else:
                print(f"Unknown line '{line}'!")
    print(f"nodes={len(graph.nodes()):5d} edges={len(graph.edges()):6d}")
    return graph
   
if False:
    import glob
    for filename in glob.glob('instances' + '/**/*.col', recursive=True):
        try:
            graph = load_graph(filename)
        except:
            print('ERROR, unable to load!')


class GraphColoringModel(OptimizerModel):

    def __init__(self, persistent_file="Z_02_Graph_Coloring.dao"):
    
        super().__init__(
            name = 'Graph Coloring',
            persistent_file = persistent_file,
            load_parameter=[
                {'name': 'mode', 'value': 'RANDOM', 'type': 'select', 'options':['RANDOM', 'FILE'],
                    'label': 'Mode', 'description': 'Create graph randomly or load from file.',
                    'on_change': [
                        'file.visible = new_value != "RANDOM"',
                        'N.visible = new_value == "RANDOM"',
                        'edge_prop.visible = new_value == "RANDOM"',
                        'random_seed.visible = new_value == "RANDOM"']},
                {'name': 'file', 'value': '', 'type': 'file_selector', 'suffix': 'col',
                    'label': 'File', 'description': 'File with suffix col for graph definition'},
                {'name': 'N', 'value': 4, 'type': 'int_slider', 'min': 1, 'max': 50, 
                    'label': 'Nodes', 'description': 'Number of nodes (vertexes)'},
                {'name': 'C', 'value': 4, 'type': 'int_slider', 'min': 1, 'max': 32, 
                    'label': 'Colors', 'description': 'Number of colors'},
                {'name': 'edge_prop', 'value': 0.5, 'type': 'float_slider', 'min': 0.0, 'max': 1.0, 'step': 0.01,
                    'label': 'Edge probability', 'description': 'Probability to have an edge conecting two nodes'},
                {'name': 'random_seed', 'value': 42, 'type': 'float_slider', 'min': 0.0, 'max': 1000.0, 'step': 0.01,
                    'label': 'Random seed', 'description': 'Seed for random graph generation'},
            ],
            build_qubo_parameter=[
                {'name': 'A', 'value': 5000.0, 'type': 'float_bounded', 'min': 0.0, 'max': 10**10, 
                    'label': 'Constraint 1 (A)', 'description': 'Scaling factor for constraint 1 (A)'},
                {'name': 'B', 'value': 500.0, 'type': 'float_bounded', 'min': 0.0, 'max': 10**10, 
                    'label': 'Constraint 2 (B)', 'description': 'Scaling factor for constraint 2 (B)'}
            ]
        )

    def load(self, mode:str, file:str, N:int, C:int, edge_prop:float, random_seed:float, silent=False):
        
        self.C = C

        if mode == 'RANDOM':
            seed(random_seed)
            self.graph = nx.Graph()     
            for i in range(N):
                self.graph.add_node(i)
            for i in range(N):
                for j in range(i):
                    if random() < edge_prop:
                        self.graph.add_edge(i, j)
        else:
            self.graph = load_graph(file)
            
        self.N = len(self.graph.nodes())
        self.E = len(self.graph.edges())
            
        if (not silent) and (mode == 'RANDOM'):
            print('Graph with %d vertices and %d edges created.' % (self.N, self.E))


    def build_qubo(self, A, B, silent=False):
    
        var_shape_set = VarShapeSet(BitArrayShape('x', (self.N, self.C), one_hot=OneHot.one_way))
        BinPol.freeze_var_shape_set(var_shape_set)

        # QUBO for unique coloring (H_1)  
        self.H_1 = BinPol()
        for n in range(self.N):
            self.H_1.add_exactly_1_bit_on([('x', n, c) for c in range(self.C)])

        # QUBO for different color on adjacent (H_2)
        self.H_2 = BinPol()
        for (m,n) in self.graph.edges():
            for c in range(self.C): 
                self.H_2.add_term(1, ('x', m, c), ('x', n, c))
                
        self.HQ = A*self.H_1 + B*self.H_2 
        
        if not silent:
            print('QUBO with %d bits created.' % self.HQ.N)

    def prep_result(self, solution_list, silent=False):
        
        solution = solution_list.min_solution
        node_color = solution['x'].data
        
        if self.C <= 6:
            colors = ['r', 'g', 'b', 'y', 'm', 'c' ]
            jcolors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0xff00ff, 0x00ffff]
        else:
            r = 1.0 / 3.0
            g = 2.0 / 5.0
            b = 1.0 / 7.0
            base = 0.5
            colors = [(base + (1-base)*(r * i % 1), 
                       base + (1-base)*(g * i % 1), 
                       base + (1-base)*(b * i % 1)) for i in range(self.C)]
            colors = [(base + (1-base)*(random()), 
                       base + (1-base)*(random()), 
                       base + (1-base)*(random())) for i in range(self.C)]
            jcolors = [int(2**24 * random()) for i in range(self.C)]

        def get_color(n):
            found = False
            for c in range(self.C):
                if node_color[n][c]:
                    found = True
                    break
            return colors[c] if found else 'w'
        
        self.colors = [get_color(n) for n in range(self.N)] 
        self.violations = [edge for edge in self.graph.edges if self.colors[edge[0]]==self.colors[edge[1]]]
        self.uncolored = [node for node in range(self.N) if self.colors[node]=='w']
        
        if not silent:
            print( "HQ  = %10.6f" % (self.HQ.compute(solution.configuration)) )
            print( "H_1 = %10.6f" % (self.H_1.compute(solution.configuration)) )
            print( "H_2 = %10.6f" % (self.H_2.compute(solution.configuration)) )

    def report(self, silent=False):

        print(('Number of colors:      {1:5d}\n' +
               'Number of nodes:       {0:5d}\n' +
               'Number of edges:       {2:5d}\n' +
               'Number of uncolored:   {3:5d}\n' +
               'Uncolored nodes:       {4:s}\n'
               'Number of violations:  {5:5d}\n' +
               'Violations:            {6:s}\n' 
              ).format(self.N, self.C, len(self.graph.edges), 
                       len(self.uncolored), str(self.uncolored), 
                       len(self.violations), str(self.violations))
             )

    def draw(self):

        if self.colors is None:
            self.colors = ['y']*len(self.graph)
        pos = nx.spring_layout(self.graph)
        fig = plt.figure(num='Graph (created at %s)' % datetime.now(), figsize=(9.0, 6.0))
        nx.draw(self.graph, pos=pos, with_labels=True, 
                nodelist=range(len(self.colors)), node_color=self.colors)
        plt.show()        


## Testing
In the following cell a test framework for the class `GraphColoring` is instantiated. Different figures for size and edge probability are used to create a random graph, which is colored by annealing.

In [ ]:
optimizer = Optimizer(GraphColoringModel())